In [ ]:
import pandas as pd
import os
import requests

def akv_query_FS_graphql(query, variable):
    try:
        response = requests.post(
            "https://api.fellesstudentsystem.no/graphql/", 
            json={"query": query, "variables": variable},
            headers={'Authorization': f'Basic {os.environ["tokenFS"]}', "Feature-Flags": "beta, experimental"},
            )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as exc:
        raise exc


spørjing = """
query MyQuery($after: String, $first: Int) {
    campuser(filter: {eierOrganisasjonskode: "0203"}) {
        pageInfo {
            endCursor
            hasNextPage
        }
        nodes {
            kode
            programStudieretter(filter: {aktivStatus: AKTIV}, after: $after, first: $first) {
                nodes {
                    student {
                        feideBruker
                    }
                }
            }
        }
    }
}
"""

In [10]:
n = 0
hentmeir = True
antal_per_side = 100
while hentmeir:
    if n == 0:
        start = None
    else:
        start = svar['data']['campuser']['pageInfo']['endCursor']
    variable = {'antal': antal_per_side, 'start': start}
    n += 1
    svar = akv_query_FS_graphql(spørjing, variable)
    # uthenting av data
    hentmeir = svar['data']['campuser']['pageInfo']['hasNextPage']
    campus_student = []
    for campus in svar['data']['campuser']['nodes']:
        campus_kode = campus['kode']
        studentar = []
        for student in campus['programStudieretter']['nodes']:
            student_kode = student['student']['feideBruker']
            campus_student.append([campus_kode, student_kode])
    hentmeir = False
        
campus_student

KeyError: 'data'

In [11]:
svar

{'errors': [{'message': "Validation error (WrongType@[campuser/nodes/programStudieretter]) : argument 'filter' with value 'ObjectValue{objectFields=[ObjectField{name='aktivStatus', value=EnumValue{name='AKTIV'}}, ObjectField{name='after', value=VariableReference{name='after'}}, ObjectField{name='first', value=VariableReference{name='first'}}]}' contains a field not in 'CampusProgramStudieretterFilterInput': 'after'",
   'locations': [{'line': 10, 'column': 33}],
   'extensions': {'classification': 'ValidationError'}}]}